In [1]:
!pwd

/Users/ashish1610dhiman/data_projects/bestbuy/notebooks/ashish_validation_train


In [2]:
import pandas as pd
import numpy as np
import pickle

In [3]:
import os
all_results = os.listdir('../../results/hmm_results/')
len(all_results)

186

### Read all HMM results

In [4]:
df_list = []
for file in all_results:
    df_file = pd.read_csv(f'../../results/hmm_results/{file}')
    df_list.append(df_file)

In [5]:
hmm_results = pd.concat(df_list)
hmm_results.shape

(1897, 8)

In [6]:
hmm_results["Encoded_SKU_ID"].nunique()

215

In [7]:
hmm_results["Encoded_SKU_ID"].drop_duplicates().nunique()

215

In [8]:
hmm_results[hmm_results.duplicated(keep=False)]

,Unnamed: 0,predicted,predicted_30,predicted_45,predicted_60,predicted_90,predicted_max,Encoded_SKU_ID
0,2022-08-01,0.0,0.0,0.00,2.00,0.00,0.00,312
1,2022-08-02,2.0,2.0,0.00,0.00,0.00,2.00,312
2,2022-08-03,2.0,1.0,0.00,1.45,2.55,0.55,312
3,2022-08-04,3.0,3.0,0.00,2.00,0.00,1.00,312
4,2022-08-05,2.0,3.0,0.00,0.00,0.00,0.00,312
...,...,...,...,...,...,...,...,...
4,2022-08-05,0.0,1.0,0.00,0.00,1.55,2.45,320
5,2022-08-06,4.0,1.0,1.00,0.45,1.00,1.00,320
6,2022-08-07,2.0,3.0,0.45,1.00,2.00,0.45,320
0,2022-08-01,0.0,0.0,0.00,0.00,0.00,0.00,210


In [9]:
hmm_results[hmm_results.Encoded_SKU_ID==312].sort_values(by=["Unnamed: 0"])

,Unnamed: 0,predicted,predicted_30,predicted_45,predicted_60,predicted_90,predicted_max,Encoded_SKU_ID
0,2022-08-01,0.0,0.0,0.0,2.00,0.00,0.00,312
126,2022-08-01,0.0,0.0,0.0,2.00,0.00,0.00,312
1,2022-08-02,2.0,2.0,0.0,0.00,0.00,2.00,312
127,2022-08-02,2.0,2.0,0.0,0.00,0.00,2.00,312
2,2022-08-03,2.0,1.0,0.0,1.45,2.55,0.55,312
128,2022-08-03,2.0,1.0,0.0,1.45,2.55,0.55,312
3,2022-08-04,3.0,3.0,0.0,2.00,0.00,1.00,312
129,2022-08-04,3.0,3.0,0.0,2.00,0.00,1.00,312
4,2022-08-05,2.0,3.0,0.0,0.00,0.00,0.00,312
130,2022-08-05,2.0,3.0,0.0,0.00,0.00,0.00,312


### List of SKU's not done yet

In [10]:
with open('../../results/final_sku_model_map.pkl', 'rb') as handle:
    sku_period_dict1 = pickle.load(handle)
    
hmm_keys = ['rmse_du_15', 'rmse_du_30', 'rmse_du_45', 'rmse_du_60', 'rmse_du_90', 'rmse_du_max']
import itertools
hmm_skus = [sku_period_dict1[sub_model] for sub_model in hmm_keys]
hmm_skus = list(itertools.chain.from_iterable(hmm_skus))

hmm_skus = list(set(hmm_skus))

In [11]:
completed_skus = hmm_results["Encoded_SKU_ID"].unique()

In [12]:
remaining_skus = set(hmm_skus) - set(completed_skus)
len(remaining_skus)

76

In [13]:
def ad_set(x):
    check = x.split("hmm_result_final_")[1].split("_")
    if len(check)>3:
        return ("_".join(check[1:]).replace(".csv",""))
    else:
        return ("_".join(check).replace(".csv",""))

In [14]:
set(map(ad_set,all_results))

{'100_final1',
 '100_to_200',
 '106_final1',
 '113_final1',
 '118_final1',
 '119_final1',
 '120_final1',
 '125_to_150',
 '175_to_200',
 '17_final1',
 '1_to_100',
 '250_to_275',
 '300_to_400',
 '33_final1',
 '38_final1',
 '50_to_100',
 '89_final1',
 '98_final1'}

### Clean up results

In [15]:
VERSION = "all2"
hmm_results.drop_duplicates().rename(columns={"Unnamed: 0":"SALES_DATE"}\
                                    ).to_csv(f"../../results/hmm_result_final_{VERSION}.csv")

In [16]:
hmm_results = hmm_results.loc[hmm_results["Encoded_SKU_ID"].isin(hmm_skus)\
                             ].drop_duplicates().rename(columns={"Unnamed: 0":"SALES_DATE"})

In [17]:
VERSION = "all"
hmm_results.to_csv(f"../../results/hmm_result_final_{VERSION}.csv")